In [1]:
import pandas as pd
from docx import Document
import os

# Agriculture + Buildings

In [2]:
def rename_region_data_files(path):
    files = os.listdir(path)

    for filename in files:
        if filename.endswith('.docx'):
            new_name = filename.split('_', 1)[-1].replace(' .docx', '.docx')
            old_file = os.path.join(path, filename)
            new_file = os.path.join(path, new_name)
            os.rename(old_file, new_file)
            print(f'Renamed: {filename} to {new_name}')

In [3]:
def extract_table_from_docx(folder_path: str, filename: str, first_batch: bool) -> pd.DataFrame:
    file_path = f"{folder_path}/{filename}.docx"
    doc = Document(file_path)
    
    tables = []
    for table in doc.tables:
        table_data = [[cell.text.strip() for cell in row.cells] for row in table.rows]
        
        if len(table_data) > 1:
            df = pd.DataFrame(table_data[1:], columns=table_data[0])
            df.set_index(df.columns[0], inplace=True)
            df = df.applymap(lambda x: x.replace("\n", "").replace("\t", "") if isinstance(x, str) else x)
            df.index = df.index.map(lambda x: x.replace("\n", "").replace("\t", "") if isinstance(x, str) else x)
            tables.append(df)
    
    related_columns = [
        'Посевная площадь всех сельскохозяйственных культур в хозяйствах всех категорий, тыс. га',
        'зерновые и зернобобовые культуры', 'картофель и овощебахчевые культуры', 'кормовые культуры',
        'крупный рогатый скот', 'в том числе коровы', 'свиньи', 'овцы и козы',
        'зерно (в весе после доработки)', 'картофель', 'овощи',
        'скот и птица на убой (в убойном весе)', 'молоко', 'яйца, млн шт.',
        'Объем работ, выполненных по виду деятельности «Строительство» (в фактически действовавших ценах), млн руб.',
        'Индекс физического объема работ, выполненных по виду деятельности «Строительство», в процентах к предыдущему году',
        'Ввод в действие жилых домов, тыс. м2 общей площади жилых помещений'
    ]

    updated_columns = [
        'Посевная площадь всех сельскохозяйственных культур в хозяйствах всех категорий, тыс. га',
        'зерновые и зернобобовые культуры, тыс. га', 
        'картофель и овощебахчевые культуры, тыс. га', 'кормовые культуры, тыс. га',
        'крупный рогатый скот, тыс. голов', 'коровы, тыс. голов', 'свиньи, тыс. голов', 'овцы и козы, тыс. голов',
        'зерно (в весе после доработки), тыс. т', 'картофель, тыс. т', 'овощи, тыс. т',
        'скот и птица на убой (в убойном весе)', 'молоко', 'яйца, млн шт.',
        'Объем работ, выполненных по виду деятельности «Строительство» (в фактически действовавших ценах), млн руб.',
        'Индекс физического объема работ, выполненных по виду деятельности «Строительство», в процентах к предыдущему году',
        'Ввод в действие жилых домов, тыс. м2 общей площади жилых помещений'
    ]
    
    year_until = '2014' if first_batch else '2020'
    
    df = pd.concat([tables[5], tables[6]])
    df.index = df.index.map(lambda index: index.replace(',', ', ').replace('  ', ' ').replace('млн.', 'млн'))
    df = df.loc[related_columns, [year for year in df.columns if year > year_until]]
    df.index = updated_columns
    df = df.T
    df['region'] = filename
    df.set_index('region', append=True, inplace=True)

    return df

In [6]:
def get_regional_data(year:str):
    folder_path = f'/Users/mansagliev/Downloads/project_data/extracted_files/Region_Subekt_{year}/1_Центральный фо'
    df = pd.DataFrame()

    for region in os.listdir(folder_path):
        if region != 'Центральный фо.docx':
            try:
                region = region.replace('.docx','')
                df = pd.concat([df, extract_table_from_docx(folder_path, region, year=='2021')])
            except:
                print(f'Error processing {region}')

    df = df.sort_index()
    return df


In [ ]:
agriculture_and_building = pd.concat([get_regional_data('2021'), get_regional_data('2023')])

In [8]:
agriculture_and_building.to_excel('agriculture_and_building.xlsx')

# Forest fire area

In [191]:
def preprocess_forest_fire_area(path:str):
    '''
    Preprocess forest fire area data, from .xls format to pandas dataframe
    '''

    fire_area = pd.read_excel(path)
    df = fire_area.T[1:].reset_index()
    df['index'] = df['index'].apply(lambda x: str(x).split('.')[0])

    columns = ['Year', 'Quarter'] + list(fire_area['Unnamed: 0'].dropna().apply(lambda x: x.strip()))
    df.columns = columns

    result = df.set_index(['Year','Quarter']).T

    return result

In [193]:
forest_fire_area = preprocess_forest_fire_area('forest_fire_area.xls')
forest_fire_area.to_excel('forest_fire_area.xlsx')